In [1]:
import os
import pandas as pd
import numpy as np

from imblearn.over_sampling import SMOTE

from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle

import sys
sys.path.append("..") 
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json

hyperparameters

In [2]:
random_seed = 42
score = 'f1_weighted'

gc config

# load data_four_features

In [3]:
path = os.getcwd()+'/../data/20122018freshwater_four_feature.csv'
data_four_features = pd.read_csv(path, na_values = np.nan)

print(data_four_features.dtypes)
print(data_four_features.shape)

pH             float64
DO(mg/l)       float64
CODMn(mg/l)    float64
NH3-N(mg/l)    float64
本周水质             int64
dtype: object
(33612, 5)


In [4]:
X = data_four_features.drop(['本周水质'], axis=1) # Series
y = data_four_features['本周水质']-1 # Series

In [5]:
print("水质分布情况:")
print(y.value_counts())
print("\n各特征类型分布情况:")
print(data_four_features.dtypes.value_counts())

水质分布情况:
1    13272
2     8797
3     5472
0     2438
5     2146
4     1487
Name: 本周水质, dtype: int64

各特征类型分布情况:
float64    4
int64      1
dtype: int64


In [6]:
data_four_features.head()

,pH,DO(mg/l),CODMn(mg/l),NH3-N(mg/l),本周水质
0,7.09,10.0,5.7,0.33,3
1,6.94,12.0,5.4,0.40,3
2,7.20,9.6,4.9,0.34,3
3,6.80,11.6,6.3,0.59,4
4,6.75,11.0,6.2,0.64,4


In [7]:
print("============ train_test_split ============")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                       stratify = y, random_state = random_seed)
print("80%% train: %d/%d, 20%% test: %d/%d" %(X_train.shape[0], X.shape[0], X_test.shape[0], X.shape[0]))

============ train_test_split ============
80% train: 26889/33612, 20% test: 6723/33612


### normalize  train data

fulfill the Na with median, then standardized the data, output type ndarray

In [8]:
clean_pipeline = Pipeline([('imputer', preprocessing.Imputer(missing_values='NaN',strategy="median")),
                           ('std_scaler', preprocessing.StandardScaler()),])
X_train = clean_pipeline.fit_transform(X_train)
X_test = clean_pipeline.fit_transform(X_test)

In [9]:
y_train = y_train.values
y_test = y_test.values

## test gc

In [10]:
with open("../pkl/CE_97661_10/CE_GCForest.pkl", "rb") as f:
    gc = pickle.load(f)
y_pred = gc.predict(X_test)

print("============= 20122018 datasets' results on test =============")
print(classification_report(y_test, y_pred, digits=4))

cm = confusion_matrix(y_test, y_pred)
i=0
acc_all = np.zeros(6)
for c in cm:
    acc_all[i] = c[i]/np.sum(c)
    print("%d accuaracy: %f" %(i+1, acc_all[i]))
    i=i+1
print("acc:", np.sum(y_test == y_pred)/y_pred.shape[0])
print('f1_weighted', f1_score(y_test, y_pred, average='weighted'))

[ 2018-12-21 00:09:34,810][cascade_classifier.transform] X_groups_test.shape=[(6723, 4)]
[ 2018-12-21 00:09:34,811][cascade_classifier.transform] group_dims=[4]
[ 2018-12-21 00:09:34,812][cascade_classifier.transform] X_test.shape=(6723, 4)
[ 2018-12-21 00:09:34,813][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(6723, 4)
[ 2018-12-21 00:09:35,000][cascade_classifier.transform] [layer=1] look_indexs=[0], X_cur_test.shape=(6723, 52)
[ 2018-12-21 00:09:35,201][cascade_classifier.transform] [layer=2] look_indexs=[0], X_cur_test.shape=(6723, 52)
[ 2018-12-21 00:09:35,391][cascade_classifier.transform] [layer=3] look_indexs=[0], X_cur_test.shape=(6723, 52)
[ 2018-12-21 00:09:35,563][cascade_classifier.transform] [layer=4] look_indexs=[0], X_cur_test.shape=(6723, 52)
[ 2018-12-21 00:09:35,740][cascade_classifier.transform] [layer=5] look_indexs=[0], X_cur_test.shape=(6723, 52)


============= 20122018 datasets' results on test =============
             precision    recall  f1-score   support

          0     0.9082    0.9939    0.9491       488
          1     0.9927    0.9759    0.9842      2655
          2     0.9835    0.9801    0.9818      1760
          3     0.9779    0.9698    0.9738      1094
          4     0.9327    0.9327    0.9327       297
          5     0.9594    0.9907    0.9748       429

avg / total     0.9770    0.9763    0.9765      6723

1 accuaracy: 0.993852
2 accuaracy: 0.975895
3 accuaracy: 0.980114
4 accuaracy: 0.969835
5 accuaracy: 0.932660
6 accuaracy: 0.990676
acc: 0.9763498438197233
f1_weighted 0.9764717843100634
